In [23]:
import sys
sys.path.append('/usr/local/share/gemini/anaconda/lib/python2.7/site-packages/')
import gemini_operations as gem_ops
import toyplot
import numpy as np
import pandas
from sklearn import linear_model, datasets, neighbors, svm, cross_validation, metrics
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from itertools import combinations
from sklearn.cross_validation import StratifiedKFold
from scipy import interp

In [2]:
"""if __name__ == "__main__":
    # Set up inputs.
    gemini_db = "/Users/hujingyuan/Downloads/jasper/lung_cancer.db"
    annotations = ["TCGA_LUAD"]

    # Get dataframe for all variants.
    df = gem_ops.get_genotypes_using_thresholds(gemini_db, annotations)

    # Some checking of the data frame.
    print "Number of variants: ", len(df)
    df = df[df['num_het'] != 0] 
    print 'Remove num_het=0'
    print 'variants: ', len(df)
    print 'TCGA: ', len(df[df['TCGA_LUAD'] == 1])
    df.to_csv("/Users/hujingyuan/lung_cancer.csv")"""

'if __name__ == "__main__":\n    # Set up inputs.\n    gemini_db = "/Users/hujingyuan/Downloads/jasper/lung_cancer.db"\n    annotations = ["TCGA_LUAD"]\n\n    # Get dataframe for all variants.\n    df = gem_ops.get_genotypes_using_thresholds(gemini_db, annotations)\n\n    # Some checking of the data frame.\n    print "Number of variants: ", len(df)\n    df = df[df[\'num_het\'] != 0] \n    print \'Remove num_het=0\'\n    print \'variants: \', len(df)\n    print \'TCGA: \', len(df[df[\'TCGA_LUAD\'] == 1])\n    df.to_csv("/Users/hujingyuan/lung_cancer.csv")'

In [3]:
# load the data from csv file
df = pandas.read_csv("/Users/hujingyuan/lung_cancer.csv")
print len(df)

3103


In [4]:
# get rid of the obvious negative variants
print 'Original Data'
print 'variants: ', len(df)
print 'TCGA: ', len(df[df['TCGA_LUAD'] == 1])

df = df[df['impact_severity'] != 'LOW']
print 'Remove impact_severity=LOW'
print 'variants: ', len(df)
print 'TCGA: ', len(df[df['TCGA_LUAD'] == 1])

df = df[df['cosmic_ids'] != 'None']
print 'Remove cosmic_ids=None'
print 'variants: ', len(df)
print 'TCGA: ', len(df[df['TCGA_LUAD'] == 1])

Original Data
variants:  3103
TCGA:  50
Remove impact_severity=LOW
variants:  1797
TCGA:  50
Remove cosmic_ids=None
variants:  384
TCGA:  50


In [5]:
# For those columns with strings, convert them to numbers
def cosmic_to_num(row):
    cosmic_val = row["cosmic_ids"]
    if cosmic_val == "None":
        return 0
    else:
        return len(cosmic_val.split(','))
df["is_cosmic"] = df.apply(cosmic_to_num, axis=1)

def imp_sev_to_num(row):
    imp_sev = row['impact_severity']
    if imp_sev == 'LOW':
        return 0
    elif imp_sev == 'MED':
        return 1
    elif imp_sev == 'HIGH':
        return 2
df['impact_severityNum'] = df.apply(imp_sev_to_num, axis=1)

def sift_pred_to_num(row):
    sift_pred = row['sift_pred']
    if sift_pred == 'None':
        return 0
    elif sift_pred == 'tolerated_low_confidence':
        return 1
    elif sift_pred == 'deleterious_low_confidence':
        return 2
    elif sift_pred == 'tolerated':
        return 3
    elif sift_pred == 'deleterious':
        return 4
df['sift_predNum'] = df.apply(sift_pred_to_num, axis=1)

def polyphen_pred_to_num(row):
    polyphen_pred = row['polyphen_pred']
    if polyphen_pred == 'None':
        return -1
    elif polyphen_pred == 'unknown':
        return 0
    elif polyphen_pred == 'benign':
        return 1
    elif polyphen_pred == 'possibly_damaging':
        return 2
    elif polyphen_pred == 'probably_damaging':
        return 3
df['polyphen_predNum'] = df.apply(polyphen_pred_to_num, axis =1)
allColumns = ['HP', 'num_het', 'mean_gt_depths', 'mean_gt_alt_depths', 'mean_gt_quals', 'is_cosmic', 
              'impact_severityNum', 'sift_predNum', "polyphen_predNum"]

In [6]:
# Convert pandas dataframe to numpy array, so it can be used for sklearn.
def buildDF(column):
    A = df[column]
    X = np.asarray(A)
    B = df['TCGA_LUAD']
    Y = np.asarray(B)
    return X, Y

In [7]:
# Load multiple Scikit-learn machine learning models
def regression(model):
    if model == 'logreg':
        clf = linear_model.LogisticRegression(C=1e5)
    elif model == 'knn_uniform':
        n_neighbors = 15
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')
    elif model == 'knn_distance':
        n_neighbors = 15
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
    elif model == 'svm_svc':
        C = 1
        clf = svm.SVC(kernel='linear', C=C)
    elif model == 'svm_rbf_svc':
        C = 1
        clf = svm.SVC(kernel='rbf', gamma=0.7, C=C)
    elif model == 'svm_poly_svc':
        C = 1
        clf = svm.SVC(kernel='poly', degree=3, C=C)
    elif model == 'svm_linear_svc':
        C = 1
        clf = svm.LinearSVC(C=C)
    elif model == 'decision_tree':
        C = 1
        clf = DecisionTreeClassifier()       
    return clf

In [8]:
# Predict the result with sklearn model(positive or negative variant)
def predict(model, column):
    X, Y = buildDF(column)
    clf = regression(model)
    # fi
    clf.fit(X, Y)  
    Y_pred = clf.predict(X)
    return Y_pred

In [9]:
# Calculate the precision and recall
def presAndRecall(model, column):
    X, Y = buildDF(column)
    clf = regression(model)
    Y_pred = predict(model, column)
    precision = precision_score(Y, Y_pred)
    recall = recall_score(Y, Y_pred)
    return [precision, recall]

In [10]:
def crossValidation(model, column):
    clf = regression(model)
    X, Y = buildDF(column)
    skf = StratifiedKFold(Y, n_folds=6)
    PrecisionAndRecall = []
    for train_index, test_index in skf:
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, Y_train)    
        Y_pred = clf.predict(X_test)
        precision = precision_score(Y_test, Y_pred)
        recall = recall_score(Y_test, Y_pred)
        PrecisionAndRecall.append((precision, recall))
    return PrecisionAndRecall

In [11]:
crossValidation('knn_distance', allColumns)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]

In [12]:
crossValidation('logreg', allColumns)

[(0.5, 0.1111111111111111),
 (0.0, 0.0),
 (0.5, 0.125),
 (0.14285714285714285, 0.375),
 (0.33333333333333331, 0.125),
 (0.66666666666666663, 0.25)]

In [13]:
crossValidation('knn_uniform', allColumns)

[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]

In [14]:
crossValidation('svm_rbf_svc', allColumns)

[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]

In [15]:
crossValidation('decision_tree', allColumns)

[(0.14285714285714285, 0.1111111111111111),
 (0.16666666666666666, 0.1111111111111111),
 (0.10000000000000001, 0.125),
 (0.125, 0.5),
 (0.0, 0.0),
 (0.27272727272727271, 0.375)]

In [13]:
presAndRecall('logreg', allColumns)

[0.5, 0.14000000000000001]

In [ ]:
allModels = ['logreg', 'knn_uniform', 'knn_distance', 'decision_tree', 'svm_rbf_svc',
             'svm_svc', 'svm_poly_svc', 'svm_linear_svc']
presRecall = []
for i in allModels:
    score = crossValidation(i, allColumns)
    presRecall.append(score)
result = pandas.DataFrame()
result['index'] = allModels
result['PresRecall'] = presRecall
print 'The first number in the array is precision, the second is recall'
print result

In [24]:
####### Generate Roc curve
def rocCurve(column):
    # load some data to play with
    X, Y = buildDF(column)
    n_samples, n_features = X.shape

    # Add noisy features
    random_state = np.random.RandomState(0)
    X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

    ###############################################################################
    # Classification and ROC analysis

    # Run classifier with cross-validation and plot ROC curves
    cv = StratifiedKFold(Y, n_folds=6)
    classifier = svm.SVC(kernel='linear', probability=True,
                         random_state=random_state)

    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, len(df))
    all_tpr = []

    for i, (train, test) in enumerate(cv):
        probas_ = classifier.fit(X[train], Y[train]).predict_proba(X[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(Y[test], probas_[:, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

    mean_tpr /= len(cv)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--',
             label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [25]:
rocCurve(['HP', 'num_het'])

In [32]:
# Visulize the regression model. (Only available with 2 variables)
def vis(model, column):
    X, Y = buildDF(df, column)
    clf, X_test, Y_test = regression(model, column)
    
    # Plot the boundaries
    minList = []
    maxList = []
    arange = []
    length = len(X[0])
    i = 0
    while i < length:
        minValue = X[:, i].min() - 1
        maxValue = X[:, i].max() + 1
        minList.append(minValue)
        maxList.append(maxValue)
        arange.append(np.arange(minValue, maxValue, 1.0))
        i += 1

    # create a meshgrid, [x_min, m_max]x[y_min, y_max]
    shape = np.meshgrid(*arange)
    ravel = [x.ravel() for x in shape]
    Z = clf.predict(np.column_stack((ravel)))
    
    # change the shape of result into meshgrid
    xx = shape[0]
    Z = Z.reshape(xx.shape)
   
    
    
    # Put the result into a color plot
    xx = shape[0]
    yy = shape[1]
    plt.figure(1, figsize=(4, 3))
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    # Plot scatterpoints
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlabel(column[0])
    plt.ylabel(column[1])

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    
    plt.show()

In [26]:
# Print out the prediction alongside the original value
def validate(model, column):
    Y_pred= predict(model, column)
    data = df[column]
    data['TCGA_LUAD'] = df['TCGA_LUAD']
    data['Predict'] = Y_pred
    data = data.sort(['Predict', 'TCGA_LUAD'], ascending = [False, False])
    return data

In [27]:
# Which two columns are good at predicting the result
listOfColumns = list(combinations(allColumns, 2))
print 'The first number in the array is precision, the second is recall'
presAndRecallDict = {}
for i in listOfColumns:    
    presAndRecallDict[i] = presAndRecall('knn_distance', list(i))
s  = pandas.Series(presAndRecallDict, index=presAndRecallDict.keys())
print s  

The first number in the array is precision, the second is recall
(mean_gt_quals, impact_severityNum)                    [1.0, 0.98]
(is_cosmic, sift_predNum)                   [0.705882352941, 0.24]
(num_het, is_cosmic)                        [0.894736842105, 0.34]
(mean_gt_depths, impact_severityNum)                   [1.0, 0.94]
(HP, mean_gt_alt_depths)                                [1.0, 0.9]
(impact_severityNum, sift_predNum)                      [0.0, 0.0]
(mean_gt_depths, sift_predNum)                         [1.0, 0.94]
(HP, impact_severityNum)                                [0.0, 0.0]
(mean_gt_alt_depths, sift_predNum)                     [1.0, 0.86]
(mean_gt_quals, polyphen_predNum)                      [1.0, 0.98]
(is_cosmic, polyphen_predNum)               [0.764705882353, 0.26]
(mean_gt_quals, is_cosmic)                              [1.0, 1.0]
(HP, is_cosmic)                             [0.722222222222, 0.26]
(HP, num_het)                                        [0.875, 0.1